<a href="https://colab.research.google.com/github/mohammed21kamall/Graduation-Project/blob/main/PartOfPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [22]:
# Define the prediction function using the provided equation
def predict_rating(ru, rv, rvi, Suv, degree_v, n_nearest_users, cu, cv):

    sum_weights = 0
    sum_rating_diff_weighted = 0

    for v in range(1, n_nearest_users + 1):
        sum_rating_diff_weighted += (rvi - rv) * Suv * w(cu, cv)  # Numerator
        sum_weights += (degree_v + 1) * abs(Suv)  # Denominator

    if sum_weights == 0:
        return 0  # Avoid division by zero

    predicted_rating = ru + (sum_rating_diff_weighted / sum_weights)
    return predicted_rating

The Similarity (Suv)

In [127]:
# Similarity
model_filename = 'drive/MyDrive/Dataset2/SimilarityOfUsers.csv'
Suv = pd.read_csv(model_filename)

Suv = Suv.clip(upper=1.0)
Suv = Suv.applymap(lambda x: max(0, x))
Suv = Suv.applymap(lambda x: x if x >= 0.4 else float(0.0))

Suv.head()

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,1.000000,0.0,0.0,0.516196,0.0,0.0,0.540215,0.0,0.0,0.0,...,0.0,0.480139,0.651735,0.000000,0.436931,0.00000,1.0,1.00000,0.000000,0.410598
1,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
2,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
3,0.516196,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,1.000000,0.000000,0.000000,0.88533,0.0,0.47253,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,1.0,1.0,0.000000,1.0,0.0,0.0,...,0.0,1.000000,0.000000,0.724984,0.000000,0.00000,0.0,0.00000,0.526469,0.000000


In [149]:
threshold=0.8
# Filter similar user IDs for a specified user ID
def get_similar_user_ids(user_id, threshold):
    # Filter the DataFrame to get similarity values for the specified user ID
    similarities_to_user = Suv[user_id]

    # Filter similar user IDs with similarity >= threshold
    similar_user_ids = similarities_to_user[similarities_to_user >= threshold].index.tolist()

    # Remove the specified user ID from the list if it's present
    similar_user_ids.remove(user_id) if user_id in similar_user_ids else None

    # Increase each user ID by 1
    similar_user_ids = [(int(user) + 1) for user in similar_user_ids]

    return similar_user_ids

# Prompt the user to enter the user ID
user_id = input("Enter the user ID: ")

# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.columns:
    print("User ID not found!")
else:
    # Get similar user IDs for the specified user ID
    similar_user_ids = list(get_similar_user_ids(user_id, threshold))

    # Display similar user IDs
    print("Similar User IDs:", similar_user_ids)


Enter the user ID: 4
Similar User IDs: [4, 57, 64, 84, 132, 135, 156, 182, 195, 198, 199, 202, 216, 221, 275, 288, 290, 330, 357, 387, 391, 412, 414, 453, 469, 474, 480, 489, 590, 597, 600, 603, 606]


In [158]:

# 0.3 is the threshold i.e the cosine similarity should be at least greater than 0.3 between two users.
n = int(input("Enter the number of similar user IDs to display: "))
print("\nSimilarity Values: \n")
print("User ID" +" "+ "Similarity")
similarity_values = Suv[Suv[user_id] >= 0.8][user_id].sort_values(ascending=False)[0:n]
print(similarity_values)


Enter the number of similar user IDs to display: 4

Similarity Values: 

User ID Similarity
3      1.0
63     1.0
83     1.0
602    1.0
Name: 4, dtype: float64


In [159]:
print(f"The Values For All Greater Users For User ID {user_id}")
similarity_values_list = similarity_values.tolist()
print(similarity_values_list)

The Values For All Greater Users For User ID 4
[1.0, 1.0, 1.0, 1.0]


In [160]:
# Extract user IDs from the index of similarity_values and append them to a list
print(f"All Greater User IDs For User ID {user_id} :")
user_ids_list = []
for index in similarity_values.index:
    user_ids_list.append(index)
print("User IDs:", user_ids_list)

All Greater User IDs For User ID 4 :
User IDs: [3, 63, 83, 602]


Rating U (ru) & Rating V (rv)

In [161]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [162]:
# Function to get rating for a user and movie
def get_rating(ratings, user_id, movie_id):
    rating = ratings[(ratings['userId'] == user_id) & (ratings['movieId'] == movie_id)]['rating'].values
    if len(rating) > 0:
        return rating[0]
    else:
        return None

# Prompt the user for input for two users and a movie
movie_id = int(input("Enter the movieId: "))
ratings_list = []

for i in range(n):
    user_id = user_ids_list[i]
    rating = get_rating(ratings, user_id, movie_id)
    ratings_list.append(rating)

# Print all ratings
print("Ratings for movie", movie_id, "are:", ratings_list)

Enter the movieId: 1
Ratings for movie 1 are: [None, 5.0, None, None]


In [16]:
ratings_list[3]

3.0

All Movies ID For User


In [163]:
# Function to get all movie IDs rated by users in a list
def get_list_rated_movie_ids(user_ids):
    rated_movies_by_users = {}  # Dictionary to store rated movies by each user

    for user_id in user_ids:
        # Filter ratings DataFrame for the specified user ID
        user_ratings = ratings[ratings['userId'] == user_id]

        # Extract and store the unique movie IDs rated by the user
        rated_movie_ids = user_ratings['movieId'].unique()
        rated_movies_by_users[user_id] = rated_movie_ids

    return rated_movies_by_users

# Example: Provide a list of user IDs
list_id = user_ids_list  # Example list of user IDs

# Get all movie IDs rated by the specified users
rated_movies_by_users = get_list_rated_movie_ids(list_id)

# Print the movie IDs rated by each user
for user_id, rated_movies in rated_movies_by_users.items():
    print(f"Movie IDs rated by User {user_id}:")
    print(rated_movies)
    print("\n")

Movie IDs rated by User 3:
[   31   527   647   688   720   849   914  1093  1124  1263  1272  1275
  1302  1371  1587  2018  2080  2090  2105  2288  2424  2851  3024  3210
  3703  3949  4518  5048  5181  5746  5764  5919  6238  6835  7899  7991
 26409 70946 72378]


Movie IDs rated by User 63:
[     1     10     32     34     47     50    110    111    150    153
    165    208    231    260    293    296    318    344    356    357
    364    367    377    480    500    527    555    586    588    589
    592    595    608    648    745    778    780    858    912    913
    924    953    969   1036   1073   1080   1084   1089   1097   1136
   1148   1193   1196   1198   1200   1201   1202   1203   1208   1210
   1213   1214   1220   1221   1222   1223   1234   1262   1265   1270
   1278   1282   1288   1291   1387   1393   1527   1580   1617   1682
   1704   1732   1923   1945   1965   1968   2000   2028   2078   2115
   2329   2542   2571   2683   2692   2716   2788   2791   2858  

In [ ]:
movie = pd.read_csv("drive/MyDrive/Dataset2/movies.csv")
movie = movie[['movieId', 'title']]
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


Mean Of Ratings (ru & rv)

In [31]:
mean_ratings_by_user = ratings.groupby('userId')['rating'].mean().reset_index()
mean_ratings_by_user.head(610)

,userId,rating
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364
...,...,...
605,606,3.657399
606,607,3.786096
607,608,3.134176
608,609,3.270270
